#### [ 한글 데이터셋 RNN ]
- 데이터셋 : Korpora

In [2]:
import pandas as pd
import numpy as np
from Korpora import Korpora

In [3]:
corpus = Korpora.load('nsmc')
corpus_df = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\K

In [4]:
trainDF = corpus_df.sample(frac=0.9, random_state=42)     # DF
testDF = corpus_df.drop(trainDF.index)                      # DF

In [5]:
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45000 entries, 33553 to 6838
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    45000 non-null  object
 1   label   45000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [6]:
print(trainDF.head(5).to_markdown())
print('Training Data Size :', len(trainDF))
print('Testing Data Size :', len(testDF))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
Training Data Size : 45000
Testing Data Size : 5000


[2-1] 토큰화 진행 => 문장 -> 단어

In [7]:
from konlpy.tag import Okt

### 토큰화 인스턴스 생성
tokenizer = Okt()

In [8]:
### 문장들을 단어 단위로 분리 (stem : 어근 처리)
train_tokens = [tokenizer.morphs(text, stem=True) for text in trainDF.text]
test_tokens = [tokenizer.morphs(text, stem=True) for text in testDF.text]

[2-2] 토큰화 => 단어/어휘 사전 생성

In [9]:
from collections import Counter

In [10]:
### 단어사전 생성 함수
def build_vocab(corpus, vocab_size, special_tokens):
    counter = Counter()
    # 단어/토큰에 대한 빈도수 계산
    for tokens in corpus:
        counter.update(tokens)
    
    # 단어/어휘 사전 생성
    vocab = special_tokens

    # 단어/어휘 사전에 빈도수가 높은 단어 추가
    for token, count in counter.most_common(vocab_size):
        vocab.append(token)
    
    return vocab

In [11]:
vocab = build_vocab(corpus=train_tokens, vocab_size=5000, special_tokens=["<PAD>", "<UNK>"])

In [12]:
print(type(vocab))

<class 'list'>


In [13]:
print(f'[VOCAB] -> {len(vocab)}개\n')
print(vocab[:10])

[VOCAB] -> 5002개

['<PAD>', '<UNK>', '.', '이', '영화', '보다', '하다', '의', '..', '에']


[3] 데이터 가공
- 토큰 데이터 정수 인코딩
- 데이터 길이 표준화 => 데이터의 길이 맞추기 (1개 문장 구성하는 단어 수 통일)

[3-1] 인코딩 & 디코딩 인덱싱

In [14]:
### 인코딩 : 문자 >>> 숫자로 변환
token_to_id = {token: idx for idx, token in enumerate(vocab)}

### 디코딩 : 숫자 >>> 문자로 변환
id_to_token = {idx: token for idx, token in enumerate(vocab)}

In [15]:
### 단어를 정수로 변환 (단어/어휘 사전에 없는 문자는 <UNK>에 대응하는 숫자[1]로 처리)
UNK_ID = token_to_id["<UNK>"]   # 1
train_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in train_tokens]
test_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in test_tokens]

[3-2] 데이터 구성 단어 수 맞추기 즉, 패딩(padding)
- 단어 수 선정 필요
- 선저오딘 단어 수에 맞게 데이터 조절 => 길면 잘라내기, 짧으면 채우기

In [55]:
### 패딩 처리 함수
def pad_sequence(sentences, max_length, pad, start='R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'R' else sen[-1 * max_length:]
        pad_length = max_length - len(sen)  # sen이 원래 max_length 보다 짧았다면 1이상의 값 저장
        padd_sen = sen + [pad]*pad_length if start == 'R' else [pad]*pad_length + sen
        result.append(padd_sen)
    return result

In [56]:
### 학습용, 테스트용 데이터 패딩 처리
PAD_ID = token_to_id.get('<PAD>')
MAX_LENGTH = 32     # 데이터를 분석해서 결정해야 한다.

train_ids = pad_sequence(train_ids, MAX_LENGTH, PAD_ID)     # 일정한 길이의 정수 인코딩 데이터
test_ids = pad_sequence(test_ids, MAX_LENGTH, PAD_ID)       # 일정한 길이의 정수 인코딩 데이터

In [60]:
print(f'[train_ids] -> {len(train_ids[11])}개')
print(f'[test_ids] -> {len(test_ids[11])}개')

[train_ids] -> 32개
[test_ids] -> 32개


In [62]:
print(train_ids[11])
print(test_ids[11])

[96, 4, 788, 305, 75, 650, 96, 125, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[59, 10, 447, 2, 80, 2, 22, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[4] 학습 준비
- Dataset, DataLoader 준비
- 학습용/테스트용 함수
- 모델 클래스
- 학습 관련 변수 => DEVICE, OPTIMIZER, MODEL 인스턴스, EPOCHS, BATCH_SIZE, LOSS_FN

In [67]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [96]:
### 데이터셋 생성 : List >>> Tensor
print(type(train_ids), type(trainDF.label.values))

dataTS1 = torch.LongTensor(train_ids)   # 임베딩에 들어가는 것은 Long, Int
labelTS1 = torch.FloatTensor(trainDF.label.values)
dataTS2 = torch.LongTensor(test_ids)    # 임베딩에 들어가는 것은 Long, Int
labelTS2 = torch.FloatTensor(testDF.label.values)

print(dataTS1.shape, labelTS1.shape)
print(dataTS2.shape, labelTS2.shape)

# 데이터셋 생성
trainDS = TensorDataset(dataTS1, labelTS1)
testDS = TensorDataset(dataTS2, labelTS2)

<class 'list'> <class 'numpy.ndarray'>
torch.Size([45000, 32]) torch.Size([45000])
torch.Size([5000, 32]) torch.Size([5000])


In [97]:
### 데이터로더 생성
BATCH_SIZE = 32

trainDL = DataLoader(trainDS, BATCH_SIZE, shuffle=True)
testDL = DataLoader(testDS, BATCH_SIZE, shuffle=True)

[4-2] 모델 클래스 정의
- 입력층 : Embedding Layer
- 은닉층 : RNN/LSTM Layer
- 은닉층 : Dropout Layer
- 출력층 : Linear Layer

In [98]:
from torch import nn

class SentenceClassifier(nn.Module):
    
    def __init__(
            self,
            vocab_size,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type='lstm'
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=0
        )

        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        
        self.classifier = nn.Linear(hidden_dim * 2, 1) if bidirectional else nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [99]:
### DEVICE, CLASSIFIER, CRITERION, OPTIMIZER
from torch import optim

vocab_size = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CLASSIFIER = SentenceClassifier(
    vocab_size=vocab_size, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
)
CRITERION = nn.BCEWithLogitsLoss().to(DEVICE)
OPTIMIZER = optim.RMSprop(CLASSIFIER.parameters(), lr=0.001)    # Adam() 써도 됨

In [103]:
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()       # 학습 모드 ON
    losses = list()     # []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)
        
        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f'Train Loss {step} : {np.mean(losses)}')

In [106]:
def test(model, datasets, criterion, device):
    model.eval()    # 테스트 모드 ON
    losses = list() # []
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) > 0.5
        corrects.extend(torch.eq(yhat, labels).cpu().tolist())
    
    print(f'Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}')

In [107]:
EPOCHS = 10
interval = 500

for epoch in range(EPOCHS):
    print(f'[EPOCH {epoch+1}]')
    train(CLASSIFIER, trainDL, CRITERION, OPTIMIZER, DEVICE, interval)
    test(CLASSIFIER, testDL, CRITERION, DEVICE)

[EPOCH 1]
Train Loss 0 : 0.13033577799797058
Train Loss 500 : 0.1405233267914065
Train Loss 1000 : 0.1470484417350544
Val Loss : 0.5373646957669288, Val Accuracy : 0.83
[EPOCH 2]
Train Loss 0 : 0.1210232824087143
Train Loss 500 : 0.12365235728462656
Train Loss 1000 : 0.12513974607251324
Val Loss : 0.560020245374388, Val Accuracy : 0.8264
[EPOCH 3]
Train Loss 0 : 0.07615429162979126
Train Loss 500 : 0.10692763110964806
Train Loss 1000 : 0.110855468873076
Val Loss : 0.6034304873578867, Val Accuracy : 0.8318
[EPOCH 4]
Train Loss 0 : 0.01829984411597252
Train Loss 500 : 0.0929033929136342
Train Loss 1000 : 0.09915173987948245
Val Loss : 0.5860543424726292, Val Accuracy : 0.8272
[EPOCH 5]
Train Loss 0 : 0.024282090365886688
Train Loss 500 : 0.08758405432339289
Train Loss 1000 : 0.09086835841284498
Val Loss : 0.650977571014386, Val Accuracy : 0.8256
[EPOCH 6]
Train Loss 0 : 0.0596318244934082
Train Loss 500 : 0.08178041104000515
Train Loss 1000 : 0.08541374114275976
Val Loss : 0.693238789678